In [7]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [8]:
train = pd.read_csv('../data/train.csv')

In [9]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
import mlflow
import numpy as np
import warnings
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor

warnings.filterwarnings('ignore')

def objective(trial):
   with mlflow.start_run(run_name=f'Trial-{trial.number}', nested=True):
       params = {
           'n_estimators': trial.suggest_int('n_estimators', 50, 500),
           'max_depth': trial.suggest_int('max_depth', 3, 20),
           'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
           'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
           'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
           'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
           'random_state': 42
       }
       
       mlflow.log_params(params)
       
       model = ExtraTreesRegressor(**params)
       
       kf = KFold(n_splits=10, shuffle=True, random_state=42)
       rmses = []
       
       for train_idx, val_idx in kf.split(X):
           X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
           y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
           
           model.fit(X_tr, y_tr)
           y_pred = model.predict(X_va)
           
           rmses.append(root_mean_squared_error(y_va, y_pred))
       
       rmse_mean = np.mean(rmses)
       mlflow.log_metric('rmse', rmse_mean)
       
       return rmse_mean

In [12]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='Extra Trees optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-28 14:23:57,014] A new study created in memory with name: no-name-91506f6a-f567-468b-bb06-3f89de693d56
[I 2025-07-28 14:23:59,267] Trial 0 finished with value: 0.7184475562306635 and parameters: {'n_estimators': 218, 'max_depth': 20, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.7184475562306635.


🏃 View run Trial-0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/b412233fe2fa4c4da4119383583a3662
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 14:24:01,564] Trial 1 finished with value: 0.7713978437120709 and parameters: {'n_estimators': 369, 'max_depth': 3, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.7184475562306635.


🏃 View run Trial-1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/95ea02e5aecd496bab7e35533f7f0610
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 14:24:04,244] Trial 2 finished with value: 0.5770234587242555 and parameters: {'n_estimators': 244, 'max_depth': 8, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True}. Best is trial 2 with value: 0.5770234587242555.


...

🏃 View run Trial-47 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/fd60dbc465d3491fb3f9094a53491993
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 14:27:29,022] Trial 48 finished with value: 0.589854913377606 and parameters: {'n_estimators': 325, 'max_depth': 17, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': None, 'bootstrap': False}. Best is trial 32 with value: 0.46032780398501083.


🏃 View run Trial-48 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/f7acd5b384bf47fb94eaf93363ec0b78
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 14:27:32,383] Trial 49 finished with value: 0.5776578489205884 and parameters: {'n_estimators': 299, 'max_depth': 18, 'min_samples_split': 12, 'min_samples_leaf': 4, 'max_features': None, 'bootstrap': True}. Best is trial 32 with value: 0.46032780398501083.


🏃 View run Trial-49 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/672a790a83204deaaa3e02028cbfe083
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
Melhores hiperparâmetros:
{'n_estimators': 353, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': False}
Melhor RMSE: 0.4603
🏃 View run Extra Trees optimization at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/8029c50fe40045a59550c1df1fa588bc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
